## Function Examples
Below are some C# implementations of various ways that function may appear in a code base

#### Typical and Lambda Functions

In [ ]:
// typical function
public static int MultiplyBy2AndAdd(int x, int y) {
    return (x * 2) + (y * 2);
}

// lambda function
var IsMod3 = (int x) => x % 3 == 0; // aka: Func<int, bool> 

Console.WriteLine($"MultiplyBy2AndAdd(2, 3): {MultiplyBy2AndAdd(2, 3)}"); // == 10
Console.WriteLine($"IsMod(9): {IsMod3(9)}"); // == true


#### Delegate

Using the `Comparison<T>` delegate type from the `System` namespace
```csharp
public namespace System
{
    public delegate Comparison<T>(T left, T right);
}
```

In [ ]:
Comparison<string> sortAlphabetically = (left, right) =>
    left.CompareTo(right);

var beerList = new List<string> { "tripel", "ipa", "sour ale", "dunkel", "lager", "aged", "stout" };
beerList.Sort(sortAlphabetically); 
beerList.Display();

#### Dictionaries

In [ ]:
var times2 = new Dictionary<int, int>
{
    { 0, 0 },
    { 1, 2 },
    { 2, 4 },
    { 3, 6 },
    { 4, 8 }
};

Console.WriteLine($"times2[3]: {times2[3]}");

### e pluribus unum: creating a higher order function

Below is sort of the canonical example that we probably use everyday using the `Where` method. `Where` takes a function that has a type `T` input and produces a `bool` (aka: `Func<T, bool>`).

In [ ]:
beerList.Where(beer => beer.StartsWith("s")).Display();

We have already defined a HOF that produces a function, it is called `IsMod3`. That function produces a function that takes an input of `int` and returns a `bool`. 

In [ ]:
Enumerable.Range(0, 30).Where(IsMod3).Display();

_Bonus_: You can get pretty crazy with passing functions and one aspect of more advanced FP is partial applications. This allows you to create functions interatively. 

In [ ]:
// we have a function that outputs some log message
var log = (string level, string message) => Console.WriteLine($"{level.ToUpper()}: {message}");
log("WARNING", "We did something bad but not terrible.");
log("Error", "THIS WAS REALLY BAD!");
log("debug", "Meh, just some random data.");

// that could get tedious, so lets add some generic log level functions that pre populate the level so
// we will rewrite the log function so it can be partially applied
var newlog = (string level) => (string message) =>  Console.WriteLine($"{level.ToUpper()}: {message}");

// specialized logging functions
var errorLog = newlog("error");
var debugLog = newlog("Debug");
var warningLog = newlog("WARNING");

// using the specialized functions
errorLog("THIS IS BAD!");
debugLog("Some random info.");
warningLog("Not terrible but you should know about it.")


## Impure Function example

This is a super simple example of some code we have either written or have come across.  In this example we have `Fermenter` class that calculates the size of the fermenter on the number of ingredents in the beer. We default to a 5 gallon size and go to a 10 gallon one with more than 5 ingredients and then up to a 15 gallon one if there are more than 10 ingredients.  

In [ ]:
// Calculates the size of fermentation vessel we need
// based on the amount of ingredients
public class Fermenter
{
    private List<string> ingredients = new();
    private string equipmentNeeded = "5 gallon fermenter";

    public void AddIngredient(string ingredient)
    {
        ingredients.Add(ingredient);
        if(ingredients.Count > 10) 
        {
            equipmentNeeded = "15 gallon fermenter";
        } 
        else if (ingredients.Count > 5) 
        {
            equipmentNeeded = "10 gallon fermenter";
        }
    }

    public string GetEquipmentNeeded() 
    {
        return equipmentNeeded;
    }

    public List<string> GetIngredients() 
    {
        return ingredients;
    }
}

// We create a new Fermenter
var fermenterCalc = new Fermenter();
fermenterCalc.AddIngredient("Cascade Hops"); // Add some hops
fermenterCalc.GetIngredients().Display(); // show what we have so far
fermenterCalc.GetEquipmentNeeded().Display(); // the size of the fermenter

var lotsOfStuff = new List<string> { "Wheat Malt", "Pilsener Malt", "Caramel Malt", "Hallertau Magnum Hops", "Safale WB40 Yeast" };
foreach(var stuff in lotsOfStuff)
{
    fermenterCalc.AddIngredient(stuff);
}

fermenterCalc.GetIngredients().Display(); // Lots of ingredients now
fermenterCalc.GetEquipmentNeeded().Display(); // bigger fermenter needed


## Cleaning up the code a bit

Based on some of the stuff we discussed, we can observe a few issues that need to be addressed. 

Here is a reminder on pure function rules (abbreviated):
1. Inputs dictate outputs
2. No side effects
3. Returns a single value

``` csharp
public class Fermenter
{
    private List<string> ingredients = new();
    private string equipmentNeeded = "5 gallon fermenter";

    public void AddIngredient(string ingredient)                            // Void method doesn't return a value
    {
        ingredients.Add(ingredient);
        if(ingredients.Count > 10) 
        {
            equipmentNeeded = "15 gallon fermenter";
        } 
        else if (ingredients.Count > 5) 
        {
            equipmentNeeded = "10 gallon fermenter";
        }                                                                   // Mutates global state
    }

    public string GetEquipmentNeeded() 
    {                                    // Uses global state
        return equipmentNeeded;
    }

    public List<string> GetIngredients() 
    {                                  // Uses global state
        return ingredients;
    }
}
```

Let's update the `GetEquipmentNeeded()` first, this returns a value based on the count of ingredients. Since we know that a pure function has it's output dictated on it's input we should pass in the state that the function needs.  

``` csharp
public string GetEquipmentNeeded(List<string> ingredients) 
{
    if (ingredients.Count > 10) 
    {
        return "15 gallon fermenter";
    } 
    else if (ingredients.Count > 5) 
    {
        return "10 gallon fermenter";
    }    
    return "5 gallon fermenter";
}
```
Usually I'm _OK_ with `if`/`ifelse` if it's simple, anything more than this I immediately will start switching to Pattern Matching:

In [ ]:
public string GetEquipmentNeeded(List<string> ingredients) => 
    ingredients switch {
        { Count: > 10 } => "15 gallon fermenter",
        { Count: > 5 } => "10 gallon fermenter",
        _ => "5 gallon fermenter"
    };


Next we can take a look at the `AddIngredients` function. This violates a few rules; it has no return value and it mutates global state. Just like in the previous refactor, lets pass in the state the function needs and return out the updated data. 

```csharp
public List<string> AddIngredients(string ingredient, List<string> ingredients) =>
    new List<string>(ingredients) { ingredient };
```
This is really just a wrapper around `List.Add` but the fuction is now pure, as there are no side effects and the inputs dictate the outputs. 

Finally we can just remove the `GetNames` function because it's not needed since there is no more global state in the class. We have now reduced the core logic down to two functions. 

Also, since we have removed the shared state and the functions are pure, we can mark the class and the functions as `static`.  

```csharp
public static class Fermenter
{
    public static List<string> AddIngredients(string ingredient, List<string> ingredients) =>
        new List<string>(ingredients) { ingredient };

    public static string GetEquipmentNeeded(List<string> ingredients) => 
        ingredients switch {
            { Count: > 10 } => "15 gallon fermenter",
            { Count: > 5 } => "10 gallon fermenter",
            _ => "5 gallon fermenter"
        };
}
```

## I/O side effects and how to deal with them

Some more quick examples of how to deal with side effects as you start to look at your code. 

Let's take, for example, some code that would act as a validator for our brewing device on when it can start the bottling process, this is usually after it has fermented for 10 - 14 days and never before that.  

So in an OO world we would define some interfaces (usually marker interfaces) and then the concrete classes (usually just one) and then all the tests to make sure it works. 

You'll then have to configure the DI to setup all the interfaces to concrete classes. 

In [ ]:
// Our state
public record Recipe (
    List<string> solids,
    double boilTemp,
    double pitchYeastTemp,
    DateTime dateToBottle 
);

// An interface for setting up the StepChecker contract
public interface IStepChecker<T>
{
    bool CanProceed(T input);
}

// The service that will abstract away DateTime.UtcNow
public interface IDateService
{
    DateTime GetDate();
}

// The concrete DateService
public record DateService : IDateService
{
    public DateTime GetDate() => DateTime.UtcNow;
}

/*
 The Implementation already simplified via record syntax. Normally you'd need a define a constructor for
 all the dependent services you want to have injected.
*/
public record BottlingStepChecker (IDateService DateService) : IStepChecker<Recipe>
{    
    private IDateService DateService { get; } = DateService;
    public bool CanProceed(Recipe recipe) => 
        (DateService.GetDate().Date >= recipe.dateToBottle.Date);
}

// using our step checker
var checker = new BottlingStepChecker(new DateService());
var pilsnerRecipe = new Recipe(default, default, default, DateTime.UtcNow.AddDays(13));

checker.CanProceed(pilsnerRecipe).Display();

There are a few options for simplifying this but the one I prefer is injecting a `Func<T>` for simple problems like this. 

In [ ]:
// refactored to accept a function.
public record BottlingStepChecker(Func<DateTime> CurrentDate) : IStepChecker<Recipe>
{
    public string GetDate() => CurrentDate().ToLongTimeString();
    public bool CanProceed(Recipe recipe) => CurrentDate().Date >= recipe.dateToBottle.Date;
}

var checker = new BottlingStepChecker(() => DateTime.UtcNow);
checker.CanProceed(pilsnerRecipe).Display();

// No need to create a new instance since the function is executed lazily, e.g. you can 
// add to DI as a singleton.
for(int i = 0; i < 6; i++)
{
    Task.Delay(500).GetAwaiter().GetResult();
    checker.GetDate().Display();
}


Now the tests are simplified as well.

```csharp
readonly DateTime fakeDate = new DateTime(2022, 1, 7);

public class FakeDateService : IDateService
{
    public DateTime GetDate() => fakeDate;
}

[Test]
public void InterfaceBasedTests()
{    
    var dtSvc = new FakeDateService(); // Moq could be used here but that's another dependency you'll rely on.
    var recipe = new Recipe(default, default, default, fakeDate.AddDays(1));    
    var sut = new BottlingStepChecker(dtSvc);
    
    sut.CanProceed(recipe).Should().BeFalse();
}

[Test]
public void FuncBasedTests()
{
    var sut = new BottlingStepChecker(() => fakeDate);
    var recipe = new Recipe(default, default, default, fakeDate.AddDays(1));

    sut.CanProceed(recipe).Should().BeFalse();
}
```

## Parallelizing pure functions

In [ ]:

static class Extensions
{
    public static string ToSentenceCase(string s)
        => s == string.Empty
            ? string.Empty
            : char.ToUpperInvariant(s[0]) + s.ToLower()[1..];
}

class ListFormatter
{
    
    int counter;
 
   string PrependCounter(string s) => $"{++counter}. {s}";
 
   public List<string> Format(List<string> list)
      => list
         .AsParallel()
         .Select(Extensions.ToSentenceCase)
         .Select(PrependCounter)
         .ToList();
}

var listOfHops = new List<string>
{
    "ADMIRAL", "Magnum", "newport", "Comet", "sorachi ace"
};

// Impure formatter.
new ListFormatter()
    .Format(listOfHops)
    .ForEach(Console.WriteLine);

```csharp
var allHops = Enumerable.Range(0, 100_000).Select(i => $"item{i}").ToList();
new ListFormatter()
    .Format(allHops)
    .ForEach(Console.WriteLine);
```

Most times, without a fast CPU or some sort of interlock you'll get an output similar to this:
```
99998. Item 89831
99999. Item 89949
99998. Item 96754
```

Refactor to pure implementations will not require the need for a lock and in some cases improve performance because of it.

```csharp
using static System.Linq.ParallelEnumerable; 

public static class ListFormatterPure
{
    public static List<string> Format(List<string> input) 
        => input.AsParallel()
            .Select(Extensions.ToSentenceCase)
            .Zip(Range(1, input.Count), (s, i) => $"{i}. {s}")
            .ToList();
}
```

### What about the absense of data?

There are times, especially with I/O, where there will not be any data that is returned. So there are specific types that we can use to help us out.

#### Unit

The `Unit` type is there to represent a function that returns no data. This differs from `void` because you can create an instance of it. You can than have your `Action<T>` functions wrapped into a `Func<Unit>` type now via an adapter function. You can still use `void` but it should be sparingly.

#### Option

The `Option` type says that you're function can take something or return something that may or may not have a value, sometimes also called `Maybe`. This can be useful in many different ways, especially in the effort of removing `null` from your code but also weird inconsistencies in libraries that you may use.

It looks like this: `Option<T> = Some<T> | None`

In [ ]:
using System.Collections.Specialized;
try
{
    var hopsAndAlphaCollection = new NameValueCollection();
    var falconerHops = hopsAndAlphaCollection["falconer"];
    Console.WriteLine("falconer hops found");

    var maltAndColorCollection = new Dictionary<string, string>();
    var caramel400 = maltAndColorCollection["caramel400"];
} 
catch( Exception ex) 
{
    Console.WriteLine(ex.GetType().Name);
}

Instead we could wrap use the `Option` type to wrap the returned values which would determine if the value is `Some<string>` or in the case of an exception, you could return a `None`. 

The net result is the caller could then recieve that value know it will be one of two types and decide what to do next, generally with some sort of pattern matching. 

For example:

```csharp
string OutputTheValue(Option<string> fromCollection) 
    => fromCollection.Match(                                        // Match would be an extension method on Option<T>
        None: () => "Value Not Found!",
        Some: (value) => Console.WriteLine($"value found: {value}")
    );
```

### Honesty is the best policy
Writing a function that doesn't lie to me or others about what it does. 

Let's say we have a calculation for IBU scores where it can't be less than 0 or more than 100 but between certain levels lives a style of beer we want to display as a general guideline. 

In [ ]:
public enum BeerStyle 
{
    Lambic,
    Amber,
    PaleAle,
    IPA,
    ImperialIPA
}

public BeerStyle GetStyleByIbu(int ibu)
{
    if(ibu < 0 || ibu > 100) 
        throw new ArgumentOutOfRangeException("IBU is invalid");
    
    return ibu switch
    {
       > 90 => BeerStyle.ImperialIPA,
       > 80 => BeerStyle.IPA,
       > 50 => BeerStyle.PaleAle,
       > 30 => BeerStyle.Amber,
       > 0 => BeerStyle.Lambic,
    };
}

GetStyleByIbu(91).Display();
GetStyleByIbu(54).Display();
GetStyleByIbu(5).Display();
GetStyleByIbu(120).Display()

In truth, the function contract should reflect that an exception can be thrown. This is best handled in a library of some kind but a simple niave implementation would look like `Either<L, R>`. The `Either` type is usually set for a Left and Right value where the Left is the error state and the Right is the value. This becomes very interesting in validation scenarios where you can have some fail and some pass and you want to report all the failures (and/or success) to the user. 

This is also called Railway Programming, because at any point in the pipeline you can switch to the error track and continue along in the pipeline. 

```csharp
public Either<BeerStyleUnknown, BeerStyle> GetStyleByIbu(int ibu)
{
    if(ibu < 0 || ibu > 100)
        return new BeerStyleUnknown("You passed in an invalid ibu reading.");
    
    return ibu switch
    {
       > 90 => BeerStyle.ImperialIPA,
       > 80 => BeerStyle.IPA,
       > 50 => BeerStyle.PaleAle,
       > 30 => BeerStyle.Amber,
       > 0 => BeerStyle.Lambic,
    };
}
```

Now when you call that function, you can pattern match on the result of the function and graceful handle any errors or continue on in your business logic, either way it's safer, concise and more readable. 
